# Select model for forecasting

ไฟล์เอกสารนี้จัดทำขึ้นเพื่อเปรียบเทียบ ประสิทธิภาพ ความแม่นยำของแต่ละโมเดล เพื่อเป็นส่วนหนึ่งในการตัดสินใจ นำไปใช้ในแอพพลิเคชั่น การทำนายหุ้น โดยโมเดลที่เราจะนำมาใช้ในการตัดสินใจมีทั้งหมด 5 โมเดล ได้แก่


1.   Prophet model 
2.   LSTM model
3.   Simple RNN model
4.   Convolutional 1D model
5.   Auto arima 

โดยที่ทั้ง 5 โมเดลจะนำไปทำนายราคาหุ้นจำนวน 5 หุ้น ที่อยู่ในตลาดของ SET100 และเราจะนำผลลัพธ์การทำนาย ไปวัดประสิทธิภาพ โดยตัดสินใจจาก MSE(mean square error), MAE(mean absolute error), RMSE(root mean square error)


# Install Package and import library

In [64]:
!pip install yfinance

In [65]:
!pip install bs4

In [66]:
!pip install pmdarima

In [67]:
from datetime import date
import yfinance as yf
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from plotly import graph_objs as go
from bs4 import BeautifulSoup 
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout,Bidirectional, SimpleRNN, Conv1D, MaxPool1D, GlobalMaxPooling1D
#from tensorflow.keras.utils import set_random_seed
from sklearn.preprocessing import StandardScaler

from pmdarima import auto_arima

เช็คสถานะ GPU

In [68]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

เก็บรายชื่อหุ้นทั้งหมด 100 หุ้นใน SET100

In [69]:
def thai_stock():
    url = "https://www.settrade.com/C13_MarketSummary.jsp?detail=SET100"
    data = requests.get(url).text
    soup = BeautifulSoup(data, "html.parser")
    selector = 'a.link-stt'
    symbols = soup.select(selector)
    set100 = list(map(lambda s: s.text, symbols[9:109]))
    set100BK = [i +".BK" for i in set100]
    return set100BK

In [70]:
thai_sc = thai_stock()

In [71]:
print(thai_sc)

กำหนดพารามิเตอร์

In [72]:
START = "2019-01-01"  #กำหนดวันที้ที่ต้องการเริ่มเก็บข้อมูลจนถึงปัจจุบัน
TODAY = date.today().strftime("%Y-%m-%d") 
test_size = 15 #จำนวนวันที่ต้องทดสอบ 
time_step = 100 #time_step คือการกำหนดตัวแปรชุดก่อนหน้าที่จะนำมาใช้เป็น X_test

# Load Data

In [73]:
def load_data(selected_stock,START, TODAY):
  data = yf.download(selected_stock, START, TODAY)
  data.reset_index(inplace=True)
  return data

# Prepare data for Prophet

In [74]:
def create_data_prophet(data, test_size):
  data1 = data[['Date', 'Close']]
  data1 = data1.rename(columns={"Date": "ds", "Close": "y"})
  train_set = data1[:len(data)-test_size].copy()
  test_set = data1[len(data)-test_size:].copy().reset_index(drop = True)
  return train_set,test_set

# Prepare data for LSTM, RNN, Conv1D model 

In [75]:
def train_test_log(data, test_size):
  df_log = data.set_index("Date")[["Close"]].copy()
  df_log['LogClose'] = np.log(df_log["Close"])
  df_log['DiffLogClose'] = df_log["LogClose"].diff()
  train = df_log.iloc[:-test_size]
  test = df_log.iloc[-test_size:]
  return train, test ,df_log 

def standard_scale(train, test, df_log):
  sc = StandardScaler()
  train_scaled = sc.fit_transform(train[['DiffLogClose']])
  test_scaled = sc.transform(test[['DiffLogClose']])
  train_idx = df_log.index <= train.index[-1]
  test_idx = df_log.index > train.index[-1]
  df_log.loc[train_idx, 'ScaledLogReturn'] = train_scaled.flatten()
  df_log.loc[test_idx, 'ScaledLogReturn'] = test_scaled.flatten()
  return df_log,sc

def create_dataset(datalog, time_step):
  datalog = datalog['ScaledLogReturn'].dropna().to_numpy()
  dataX, dataY = [],[]
  for i in range(len(datalog)-time_step):
    dataX.append(datalog[i:(i+time_step)])                       
    dataY.append(datalog[i+time_step])  
  dataX = np.array(dataX)
  dataY = np.array(dataY)
  return dataX, dataY

def split_train_test_xy_log(X_log,y_log, test_size):
  X_train_log, y_train_log = X_log[:-test_size], y_log[:-test_size]
  X_test_log, y_test_log = X_log[-test_size:], y_log[-test_size:]
  X_train_log = X_train_log.reshape(X_train_log.shape[0], X_train_log.shape[1],1 ) 
  X_test_log = X_test_log.reshape(X_test_log.shape[0], X_test_log.shape[1], 1)
  return X_train_log, y_train_log,X_test_log, y_test_log

# Prophet model

In [76]:
def prophet_model(train_set, test_set, data):
  m = Prophet()
  m.fit(train_set)
  forecast = m.predict(test_set)
  res_data = data[["Date","Close"]]
  res_y = forecast[["ds", "yhat"]]
  res_data = pd.merge(res_data, res_y, how='outer', left_on="Date",right_on="ds")
  res_data = res_data.set_index("Date")
  res_data = res_data.drop(columns = "ds",axis = 0)
  return res_data

# LSTM model 

In [77]:
def LSTM_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step): 
  tf.keras.backend.clear_session()
  tf.random.set_seed(42)
  model_LSTM = Sequential()
  model_LSTM.add(LSTM(128, return_sequences=True))
  model_LSTM.add(LSTM(64))
  model_LSTM.add(Dense(1))
  model_LSTM.compile(loss='mse', optimizer='adam')
  history = model_LSTM.fit(X_train_log, y_train_log, validation_data=(X_test_log, y_test_log), epochs=50, batch_size=128, verbose=0)
  return history,model_LSTM

# RNN model

In [78]:
def RNN_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step):
  tf.keras.backend.clear_session()
  tf.random.set_seed(42)
  model_RNN = Sequential()
  model_RNN.add(SimpleRNN(128, return_sequences=True))
  model_RNN.add(SimpleRNN(64))
  model_RNN.add(Dense(1))
  model_RNN.compile(loss='mse', optimizer='adam')
  history = model_RNN.fit(X_train_log, y_train_log, validation_data=(X_test_log, y_test_log), epochs=50, batch_size=128, verbose=0)
  return history,model_RNN

# Conv1D model

In [79]:
def CNN_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step):
  tf.keras.backend.clear_session()
  tf.random.set_seed(42)
  model_CNN = Sequential()
  model_CNN.add(SimpleRNN(128, return_sequences=True))
  model_CNN.add(SimpleRNN(64))
  model_CNN.add(Dense(1))
  model_CNN.compile(loss='mse', optimizer='adam')
  history = model_CNN.fit(X_train_log, y_train_log, validation_data=(X_test_log, y_test_log), epochs=50, batch_size=128, verbose=0)
  return history,model_CNN

# Auto arima model

In [80]:
def auto_arima_model(train_set, test_set, data):
  training = train_set ['y']
  validation = test_set ['y']
  model = auto_arima(training, start_p=1, start_q=1,max_p=3, max_q=3, m=12,start_P=0,d=1, D=1, trace=False,error_action='ignore',suppress_warnings=True)
  model.fit(training)
  forecast = model.predict(n_periods=len(validation))
  arima_result = data.copy()
  arima_result["yhat"] = np.nan
  arima_result.loc[len(arima_result) - test_size:,"yhat"] = forecast
  return arima_result

# Prediction Deep Learning model 

In [81]:
def predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model, sc):
    # multi-step forecast
    multistep_predictions = []
    last_x = X_test_log[0]
    while len(multistep_predictions) < len(X_test_log):
        p = model.predict(last_x.reshape(1, -1, 1))[0]
        multistep_predictions.append(p)
        last_x = np.roll(last_x, -1)
        last_x[-1] = p
    multistep_predictions = np.array(multistep_predictions)
    multistep_predictions = sc.inverse_transform(multistep_predictions.reshape(-1, 1)).flatten()

  #predict
    train_predict_log = model.predict(X_train_log)
    test_predict_log =model.predict(X_test_log)
    train_predict_log = sc.inverse_transform(train_predict_log)
    test_predict_log = sc.inverse_transform(test_predict_log)

  #Create_result_dataframe
    df_result = df_log.copy()
    df_result['ShiftedLogClose'] = df_result['LogClose'].shift(1)
    prev = df_result['ShiftedLogClose']
    train_idx_log = df_result.iloc[time_step + 1:-test_size].index
    test_idx_log = df_result.iloc[-test_size:].index
    df_result.loc[train_idx_log, 'LSTM_1step_train'] = prev[train_idx_log] + train_predict_log.flatten()
    df_result.loc[test_idx_log, 'LSTM_1step_test'] = prev[test_idx_log] + test_predict_log.flatten()
    df_result.loc[test_idx_log, 'multistep'] = train.iloc[-1]['LogClose'] + np.cumsum(multistep_predictions)
    list_inv = ['LSTM_1step_train', 'LSTM_1step_test', 'multistep']
    for i in list_inv :
      df_result[i + "_inv"] = np.exp(df_result[i])
    return df_result

# Calculate Score Prophet and Arima

In [82]:
def cal_score(df_result, test_size, selected_stock,scores_model,name_model):
  df_eva = df_result.iloc[-test_size:, :]
  mae = mean_absolute_error(df_eva["Close"], df_eva["yhat"]) 
  mse = mean_squared_error(df_eva["Close"], df_eva["yhat"], squared = True)
  rmse = mean_squared_error(df_eva["Close"], df_eva["yhat"], squared = False)
  scores_model.append({'Model': name_model,'MAE': mae, 'MSE': mse,'RMSE': rmse, 
                   "Stock": selected_stock})
  return scores_model

# Calculate Score Deep learning (LSTM, RNN, CNN)

In [83]:
def cal_score_dl(df_result, test_size, scores_model, selected_stock, name_model):
  #MSE_test = mean_squared_error(df_result.loc[test_idx_log, 'Close'], df_result.loc[test_idx_log, 'LSTM_1step_test_inv'])
  #RMSE_test = math.sqrt(MSE_test)
  df_result = df_result.dropna(axis = 0, subset = ["multistep_inv"])
  y_true = df_result["Close"].tolist()
  #y_true1 = y_true[-test_size:]
  y_pred = df_result["multistep_inv"].tolist()
  #y_pred1 = y_pred[-test_size:]
  mse = mean_squared_error(y_true, y_pred, squared = True)
  mae = mean_absolute_error(y_true, y_pred) 
  rmse = math.sqrt(mse)
  scores_model.append({'Model': name_model,'MAE': mae, 'MSE': mse ,'RMSE': rmse,
                    "Stock": selected_stock})
  return scores_model

เมื่อเราสร้างฟังก์ชั้่นหลักเสร็จเรียบร้อยแล้วจะนำฟังก์ชั่นเหล่านี้มาทำนาย หุ้นเป็นจำนวน 5 หุ้น

# หุ้นที่ 1 = AOT 

## Main Function 

In [84]:
selected_stock = "AOT.BK" 
def predict_stock_lstm(selected_stock, START, TODAY, time_step):
  data =  load_data(selected_stock,START, TODAY)
  train, test ,df_log = train_test_log(data,test_size)
  df_log,sc =  standard_scale(train, test, df_log)
  X_log, y_log = create_dataset(df_log, time_step)
  X_train_log, y_train_log,X_test_log, y_test_log = split_train_test_xy_log(X_log,y_log,test_size)

  #Prophet
  train_set,test_set = create_data_prophet(data, test_size)
  df_result_Prophet = prophet_model(train_set, test_set, data)

  #LSTM 
  history,model_LSTM = LSTM_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_LSTM = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_LSTM, sc)

  #RNN
  history,model_RNN = RNN_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_RNN = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_RNN, sc) 

  #Conv1D
  history,model_CNN = CNN_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_CNN = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_CNN, sc) 

  #Auto Arima 
  arima_result = auto_arima_model(train_set, test_set, data)
  
  return df_result_Prophet, df_result_LSTM, df_result_RNN, df_result_CNN, arima_result

In [85]:
df_result_Prophet, df_result_LSTM, df_result_RNN, df_result_CNN, arima_result = predict_stock_lstm(selected_stock, START, TODAY, time_step)

### แสดงกราฟผลลัพธ์แต่ละโมเดล

In [86]:
df_result_Prophet[['Close', 'yhat']].iloc[-50:].plot(figsize=(15, 5))

In [87]:
df_result_LSTM[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [88]:
df_result_RNN[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [89]:
df_result_CNN[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [90]:
arima_result[['Close', 'yhat']].iloc[-50:].plot(figsize=(15, 5))

### Evaluate model 

In [91]:
scores_model = []

# Prophet score 
scores_model_Prophet = cal_score(df_result_Prophet, test_size, selected_stock,scores_model,name_model = "Prophet")

# Auto Arima 
scores_model_Arima = cal_score(arima_result, test_size, selected_stock,scores_model,name_model = "Arima")

#LSTM 
scores_model_LSTM = cal_score_dl(df_result_LSTM, test_size ,scores_model, selected_stock, name_model = "LSTM")

#RNN
scores_model_RNN = cal_score_dl(df_result_RNN, test_size, scores_model, selected_stock, name_model = "RNN")

#CNN
scores_model_CNN = cal_score_dl(df_result_CNN, test_size, scores_model, selected_stock, name_model = "CNN")

# หุ้นที่ 2 = SCC

### Main Function 

In [92]:
selected_stock = "SCC.BK" 
def predict_stock_lstm(selected_stock, START, TODAY, time_step):
  data =  load_data(selected_stock,START, TODAY)
  train, test ,df_log = train_test_log(data,test_size)
  df_log,sc =  standard_scale(train, test, df_log)
  X_log, y_log = create_dataset(df_log, time_step)
  X_train_log, y_train_log,X_test_log, y_test_log = split_train_test_xy_log(X_log,y_log,test_size)

  #Prophet
  train_set,test_set = create_data_prophet(data, test_size)
  df_result_Prophet = prophet_model(train_set, test_set, data)

  #LSTM 
  history,model_LSTM = LSTM_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_LSTM = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_LSTM, sc)

  #RNN
  history,model_RNN = RNN_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_RNN = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_RNN, sc) 

  #Conv1D
  history,model_CNN = CNN_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_CNN = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_CNN, sc) 

  #Auto Arima 
  arima_result = auto_arima_model(train_set, test_set, data)
  
  return df_result_Prophet, df_result_LSTM, df_result_RNN, df_result_CNN, arima_result

In [93]:
df_result_Prophet, df_result_LSTM, df_result_RNN, df_result_CNN, arima_result = predict_stock_lstm(selected_stock, START, TODAY, time_step)

In [94]:
df_result_Prophet[['Close', 'yhat']].iloc[-50:].plot(figsize=(15, 5))

In [95]:
df_result_LSTM[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [96]:
df_result_RNN[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [97]:
df_result_CNN[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [98]:
arima_result[['Close', 'yhat']].iloc[-50:].plot(figsize=(15, 5))

### Evaluate model

In [99]:
# Prophet score 
scores_model_Prophet = cal_score(df_result_Prophet, test_size, selected_stock,scores_model,name_model = "Prophet")

# Auto Arima 
scores_model_Arima = cal_score(arima_result, test_size, selected_stock,scores_model,name_model = "Arima")

#LSTM 
scores_model_LSTM = cal_score_dl(df_result_LSTM, test_size ,scores_model, selected_stock, name_model = "LSTM")

#RNN
scores_model_RNN = cal_score_dl(df_result_RNN, test_size, scores_model, selected_stock, name_model = "RNN")

#CNN
scores_model_CNN = cal_score_dl(df_result_CNN, test_size, scores_model, selected_stock, name_model = "CNN")

# หุ้นที่ 3 = PLANB

### Main Function 

In [100]:
selected_stock = "PLANB.BK" 
def predict_stock_lstm(selected_stock, START, TODAY, time_step):
  data =  load_data(selected_stock,START, TODAY)
  train, test ,df_log = train_test_log(data,test_size)
  df_log,sc =  standard_scale(train, test, df_log)
  X_log, y_log = create_dataset(df_log, time_step)
  X_train_log, y_train_log,X_test_log, y_test_log = split_train_test_xy_log(X_log,y_log,test_size)

  #Prophet
  train_set,test_set = create_data_prophet(data, test_size)
  df_result_Prophet = prophet_model(train_set, test_set, data)

  #LSTM 
  history,model_LSTM = LSTM_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_LSTM = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_LSTM, sc)

  #RNN
  history,model_RNN = RNN_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_RNN = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_RNN, sc) 

  #Conv1D
  history,model_CNN = CNN_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_CNN = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_CNN, sc) 

  #Auto Arima 
  arima_result = auto_arima_model(train_set, test_set, data)
  
  return df_result_Prophet, df_result_LSTM, df_result_RNN, df_result_CNN, arima_result

In [101]:
df_result_Prophet, df_result_LSTM, df_result_RNN, df_result_CNN, arima_result = predict_stock_lstm(selected_stock, START, TODAY, time_step)

In [102]:
df_result_Prophet[['Close', 'yhat']].iloc[-50:].plot(figsize=(15, 5))

In [103]:
df_result_LSTM[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [104]:
df_result_RNN[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [105]:
df_result_CNN[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [106]:
arima_result[['Close', 'yhat']].iloc[-50:].plot(figsize=(15, 5))

### Evaluate model

In [107]:
# Prophet score 
scores_model_Prophet = cal_score(df_result_Prophet, test_size, selected_stock,scores_model,name_model = "Prophet")

# Auto Arima 
scores_model_Arima = cal_score(arima_result, test_size, selected_stock,scores_model,name_model = "Arima")

#LSTM 
scores_model_LSTM = cal_score_dl(df_result_LSTM, test_size ,scores_model, selected_stock, name_model = "LSTM")

#RNN
scores_model_RNN = cal_score_dl(df_result_RNN, test_size, scores_model, selected_stock, name_model = "RNN")

#CNN
scores_model_CNN = cal_score_dl(df_result_CNN, test_size, scores_model, selected_stock, name_model = "CNN")

# หุ้นที่ 4 = KTB

### Main Function 

In [108]:
selected_stock = "KTB.BK" 
def predict_stock_lstm(selected_stock, START, TODAY, time_step):
  data =  load_data(selected_stock,START, TODAY)
  train, test ,df_log = train_test_log(data,test_size)
  df_log,sc =  standard_scale(train, test, df_log)
  X_log, y_log = create_dataset(df_log, time_step)
  X_train_log, y_train_log,X_test_log, y_test_log = split_train_test_xy_log(X_log,y_log,test_size)

  #Prophet
  train_set,test_set = create_data_prophet(data, test_size)
  df_result_Prophet = prophet_model(train_set, test_set, data)

  #LSTM 
  history,model_LSTM = LSTM_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_LSTM = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_LSTM, sc)

  #RNN
  history,model_RNN = RNN_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_RNN = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_RNN, sc) 

  #Conv1D
  history,model_CNN = CNN_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_CNN = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_CNN, sc) 

  #Auto Arima 
  arima_result = auto_arima_model(train_set, test_set, data)
  
  return df_result_Prophet, df_result_LSTM, df_result_RNN, df_result_CNN, arima_result

In [109]:
df_result_Prophet, df_result_LSTM, df_result_RNN, df_result_CNN, arima_result = predict_stock_lstm(selected_stock, START, TODAY, time_step)

In [110]:
df_result_Prophet[['Close', 'yhat']].iloc[-50:].plot(figsize=(15, 5))

In [111]:
df_result_LSTM[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [112]:
df_result_RNN[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [113]:
df_result_CNN[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [114]:
arima_result[['Close', 'yhat']].iloc[-50:].plot(figsize=(15, 5))

### Evaluate model

In [115]:
# Prophet score 
scores_model_Prophet = cal_score(df_result_Prophet, test_size, selected_stock,scores_model,name_model = "Prophet")

# Auto Arima 
scores_model_Arima = cal_score(arima_result, test_size, selected_stock,scores_model,name_model = "Arima")

#LSTM 
scores_model_LSTM = cal_score_dl(df_result_LSTM, test_size ,scores_model, selected_stock, name_model = "LSTM")

#RNN
scores_model_RNN = cal_score_dl(df_result_RNN, test_size, scores_model, selected_stock, name_model = "RNN")

#CNN
scores_model_CNN = cal_score_dl(df_result_CNN, test_size, scores_model, selected_stock, name_model = "CNN")

# หุ้น 5 = KBANK.BK

### Main Function 

In [116]:
selected_stock = "KBANK.BK" 
def predict_stock_lstm(selected_stock, START, TODAY, time_step):
  data =  load_data(selected_stock,START, TODAY)
  train, test ,df_log = train_test_log(data,test_size)
  df_log,sc =  standard_scale(train, test, df_log)
  X_log, y_log = create_dataset(df_log, time_step)
  X_train_log, y_train_log,X_test_log, y_test_log = split_train_test_xy_log(X_log,y_log,test_size)

  #Prophet
  train_set,test_set = create_data_prophet(data, test_size)
  df_result_Prophet = prophet_model(train_set, test_set, data)

  #LSTM 
  history,model_LSTM = LSTM_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_LSTM = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_LSTM, sc)

  #RNN
  history,model_RNN = RNN_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_RNN = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_RNN, sc) 

  #Conv1D
  history,model_CNN = CNN_model(X_train_log,y_train_log,X_test_log,y_test_log, time_step)
  df_result_CNN = predict_result(X_train_log, X_test_log, df_log, time_step, test_size, train, model_CNN, sc) 

  #Auto Arima 
  arima_result = auto_arima_model(train_set, test_set, data)
  
  return df_result_Prophet, df_result_LSTM, df_result_RNN, df_result_CNN, arima_result

In [117]:
df_result_Prophet, df_result_LSTM, df_result_RNN, df_result_CNN, arima_result = predict_stock_lstm(selected_stock, START, TODAY, time_step)

In [118]:
df_result_Prophet[['Close', 'yhat']].iloc[-50:].plot(figsize=(15, 5))

In [119]:
df_result_LSTM[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [120]:
df_result_RNN[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [121]:
df_result_CNN[['Close', 'LSTM_1step_train_inv', 'LSTM_1step_test_inv', 'multistep_inv']].iloc[-50:].plot(figsize=(15, 5))

In [122]:
arima_result[['Close', 'yhat']].iloc[-50:].plot(figsize=(15, 5))

### Evaluate model

In [123]:
# Prophet score 
scores_model_Prophet = cal_score(df_result_Prophet, test_size, selected_stock,scores_model,name_model = "Prophet")

# Auto Arima 
scores_model_Arima = cal_score(arima_result, test_size, selected_stock,scores_model,name_model = "Arima")

#LSTM 
scores_model_LSTM = cal_score_dl(df_result_LSTM, test_size ,scores_model, selected_stock, name_model = "LSTM")

#RNN
scores_model_RNN = cal_score_dl(df_result_RNN, test_size, scores_model, selected_stock, name_model = "RNN")

#CNN
scores_model_CNN = cal_score_dl(df_result_CNN, test_size, scores_model, selected_stock, name_model = "CNN")

# Result

In [124]:
results = pd.DataFrame(scores_model).sort_values(by='RMSE',ascending=True)
results

In [125]:
result_g = results.groupby(by = "Model").mean()
result_g.sort_values(by='RMSE',ascending=True)